In [1]:
import pandas as pd
import json
import os
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
import spacy


In [2]:
nlp = spacy.load("en_core_web_sm")


In [5]:
project_path = os.getcwd()
generated = pd.read_csv(os.path.join(project_path, "..", "..", "output", "few_shot_summaries", "test_generated.csv"))
generated.columns


Index(['discharge_report', 'discharge_summary', 'generated_summary'], dtype='object')

In [6]:
original_summaries_sentences = []
generated_summaries_sentences = []
for i, row in generated.iterrows():
    original_doc = nlp(row["discharge_summary"])
    original_summaries_sentences.append([sent.text for sent in original_doc.sents])
    generated_doc = nlp(row["generated_summary"])
    generated_summaries_sentences.append([sent.text for sent in generated_doc.sents])


In [16]:
def write_sent_lines(summaries, file_path):
    for summary in summaries:
        with open(file_path, "a") as f:
            f.write("###SUMMARY: \n")
            for sent in summary:
                f.write(sent + "\n")
            f.write("\n\n")




In [17]:
write_sent_lines(original_summaries_sentences, os.path.join(project_path, "..", "..", "output", "few_shot_summaries", "original_summaries_sentences.txt"))
write_sent_lines(generated_summaries_sentences, os.path.join(project_path, "..", "..", "output", "few_shot_summaries", "generated_summaries_sentences.txt"))

In [3]:
# min, max, mean, std of the length of the discharge reports and summaries
lengths = train_df['discharge_report'].str.len()  # Calculate the length of each report

# Calculate min, max, mean, and std
min_length = lengths.min()
max_length = lengths.max()
mean_length = lengths.mean()
std_length = lengths.std()

# Print the results
print(f"Min length report: {min_length}")
print(f"Max length report: {max_length}")
print(f"Mean length report: {mean_length}")
print(f"Standard deviation report: {std_length}")

lengths = train_df['discharge_summary'].str.len()  # Calculate the length of each summary

# Calculate min, max, mean, and std
min_length = lengths.min()
max_length = lengths.max()
mean_length = lengths.mean()
std_length = lengths.std()

# Print the results
print(f"Min length summary: {min_length}")
print(f"Max length summary: {max_length}")
print(f"Mean length summary: {mean_length}")
print(f"Standard deviation summary: {std_length}")


Min length report: 3416
Max length report: 33209
Mean length report: 11007.517482517482
Standard deviation report: 4990.852398247897
Min length summary: 576
Max length summary: 2210
Mean length summary: 1145.2202797202797
Standard deviation summary: 322.84188255421503


In [12]:
output_path = os.path.join(project_path, '..', '..', 'output', 'zero_shot_summaries')
df_generated = pd.read_csv(os.path.join(output_path, 'test_generated.csv'))


In [13]:
df_generated.columns

Index(['discharge_report', 'discharge_summary', 'generated_summary'], dtype='object')

In [15]:
df_generated['discharge_summary'][0]

"The patient is a 51-year-old male who underwent a living non-related renal transplant due to end-stage renal disease (ESRD) caused by medication toxicity and chronic dehydration. He has a history of Crohn's disease, which was treated with total colectomy and ileoanal pullthrough 20 years ago. He has been on hemodialysis since then. The patient has allergies to sulfa, penicillins, and Asacol. His past medical history includes ESRD, interstitial nephritis, and Asacol on HD via RIJ tunnel cath. \n\nDuring his hospital stay, the patient received induction immunosuppression and underwent a successful transplant with a living donor. He experienced some complications, including decreased urine output and low blood pressure, which were managed with fluid replacement and medication adjustments. The patient was discharged with a stable condition and was instructed to follow up with the transplant office and to have regular lab tests. His medications at discharge included acetaminophen, dapsone,

In [16]:
df_generated['generated_summary'][0]


" \nThe patient is a 51-year-old male with a history of Crohn's disease and end-stage renal disease (ESRD) who underwent a living non-related renal transplant. He received induction immunosuppression and was started on Prograf, Cellcept, and Solu-Medrol. Postoperatively, he experienced a decrease in urine output, which was managed with fluid replacements and a decrease in Prograf dose. He was also started on Dapsone for PCP prophylaxis due to a sulfa allergy. He was discharged on a regimen of immunosuppressive medications, including Tacrolimus, Mycophenolate Mofetil, and Valganciclovir, as well as pain medication and anti-diarrheal medication. He was instructed to follow up with the transplant office and to have labs drawn regularly. He was also advised to avoid heavy lifting, straining, and driving while taking pain medication.\nThe final answer is: There is no final answer to this problem as it is a discharge report summary. However, the key points from the report are:\n\n* The patie